Created by Bekbol Sultankulov, Data scientist

#  parsing arbuz.kz

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\91.0.4472.101\\chromedriver.exe')
driver.maximize_window()
driver.get("https://arbuz.kz/")
time.sleep(5)
driver.find_element_by_xpath('//*[@id="layout"]/div/div[1]/div/div/button/span').click()
data={}
for i in product_names:
    try:
        searchBox=driver.find_element_by_xpath('//*[@id="layout"]/main/header/div[2]/div/div[3]/div/div[1]/form/input') 
        searchBox.clear()
        searchBox.send_keys(str(i))
        searchBox.send_keys(Keys.RETURN)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="layout"]/main/section[2]/div[2]/div/div[1]/div/div/article/div/div[1]/div/div/div[1]/a').click()
        html = driver.page_source
        soup = BeautifulSoup(html)
        a=''
        for tag in soup.find_all('ol'):
            a=tag.text
        data[i]=a
    except ElementClickInterceptedException:
        driver.find_element_by_xpath('//*[@id="layout"]/div[2]/div/div/button/span').click()
        driver.find_element_by_xpath('//*[@id="layout"]/main/section[2]/div[2]/div/div[1]/div/div/article/div/div[1]/div/div/div[1]/a').click()
        html = driver.page_source
        soup = BeautifulSoup(html)
        a=''
        for tag in soup.find_all('ol'):
            a=tag.text
        data[i]=a
    except NoSuchElementException:
        data[i]='Not found'        
        continue
    

# Finding new barcodes by parsing shtrih-kody.ru/

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path='C:\\Users\\svnduw\\.wdm\\drivers\\chromedriver\\win32\\93.0.4577.15\\chromedriver.exe',chrome_options=options)
driver.get("https://shtrih-kody.ru/")
time.sleep(2)
barcode=[]
name=[]
category=[]
brand=[]
#country=[]
for i in range(len(barcodes)):
    try:        
        inputElems = driver.find_element_by_css_selector('input[name=txt]')
        inputElems.clear()
        inputElems.send_keys(str(barcodes[i]))
        inputElems.send_keys(Keys.ENTER)
        time.sleep(1)
        count = driver.find_elements_by_xpath('/html/body/div/p')
        if len(count)==4:            
            barcode.append(barcodes[i])
            name.append((count[0].text).split(':')[1])
            category.append((count[1].text).split(':')[1])
            brand.append('Not given')
        elif len(count)==5:
            barcode.append(barcodes[i])
            name.append((count[0].text).split(':')[1])
            category.append((count[1].text).split(':')[1])
            brand.append((count[2].text).split(':')[1])
        else:
            barcode.append(barcodes[i])
            name.append('Not Found')
            category.append('Not Found')
            brand.append('Not Found')                     
    except NoSuchElementException:                
        continue
        
driver.close()
        
      


In [ ]:
d={'Barcode':barcode,'Name':name,'Brand':brand,'Category':category}
df=pd.DataFrame(d)  
df=df[~df['Category'].isin(['Not Found'])]
new=df['Category'].str.split('/',n=-2, expand=True).add_prefix('Sub-category- ')
df['Category']=new['Sub-category- 0']
df=pd.concat([df,new],axis=1)
df.drop('Sub-category- 0',axis=1,inplace=True)

In [ ]:
df.drop('Sub-category- 4',axis=1,inplace=True)